In [3]:
import yaml
import os
import pprint
import pandas as pd
from pathlib import Path
from influence_moo.config import load_config

In [4]:
base_config_dir = '~/influence-multi-objective/example_sweep/base_config.yaml'
sweep_config_dir = '~/influence-multi-objective/example_sweep/sweep.yaml'

# Load in the base config
base_config = load_config(base_config_dir)

# Load in sweep config
sweep_config = load_config(sweep_config_dir)

In [ ]:
"""This uses a not in-place version of merge_dicts"""

def merge_dicts(dict1, dict2):
    result = dict1.copy()  # Make a copy of the first dictionary to avoid modifying the original
    for key, value in dict2.items():
        if key in result and isinstance(result[key], dict) and isinstance(value, dict):
            # If both values are dictionaries, merge them recursively
            result[key] = merge_dicts(result[key], value)
        else:
            # Otherwise, update/overwrite with the value from dict2
            result[key] = value
    return result


def expand_keys(dict_):
    """Expand the keys of a given dict from ['key1:key2:key3'] to ['key1']['key2']['key3'] in-place"""
    # print('dict_', dict_)
    keys = [k for k in dict_]
    print('keys', keys)
    for key in keys:
        print('key', key)
        if ":" in key:
            print('found :', key)
            keys = key.split(":")
            print('keys', keys)
            first_key = keys[0]
            print('first_key', first_key, ", dict_", dict_)
            if first_key in dict_:
                dict_[first_key] = merge_dicts(dict_[first_key], expand_keys({":".join(keys[1:]) : dict_[key]}))
            else:
                dict_[first_key] = expand_keys({":".join(keys[1:]) : dict_[key]})
            print('dict_[first_key]', dict_[first_key])
            del dict_[key]
    return dict_

e = {'a:b:c': 'd', 'a:b:f': 0}
print(e)
expand_keys(e)
print(e)
# print(ed)

In [100]:
"""This does everything in place"""

def merge_dicts(dict1, dict2):
    """Merge dict2 into dict1 recursively (ie: any nested dictionaries). For any conflicts, dict2 overwrites dict1"""
    # result = dict1.copy()  # Make a copy of the first dictionary to avoid modifying the original
    keys = [k for k in dict2]
    for key in dict2:
        if key in dict1 and isinstance(dict1[key], dict) and isinstance(dict2[key], dict):
            # If both values are dictionaries, merge them recursively
            merge_dicts(dict1[key], dict2[key])
        else:
            # Otherwise, update/overwrite with the value from dict2
            dict1[key] = dict2[key]

def expand_keys(dict_):
    """Expand the keys of a given dict from ['key1:key2:key3'] to ['key1']['key2']['key3'] in-place"""
    # print('dict_', dict_)
    keys = [k for k in dict_]
    print('keys', keys)
    for key in keys:
        print('key', key)
        if ":" in key:
            print('found :', key)
            keys = key.split(":")
            print('keys', keys)
            first_key = keys[0]
            print('first_key', first_key, ", dict_", dict_)
            new_dict = expand_keys({":".join(keys[1:]) : dict_[key]})
            if first_key in dict_:
                merge_dicts(dict_[first_key], new_dict)
            else:
                dict_[first_key] = expand_keys({":".join(keys[1:]) : dict_[key]})
            print('dict_[first_key]', dict_[first_key])
            del dict_[key]
        elif isinstance(dict_[key], dict):
            expand_keys(dict_[key])
    return dict_

e = {'a:b:c': 'd', 'a:b:f': 0}
print(e)
expand_keys(e)
print(e)
# print(ed)

{'a:b:c': 'd', 'a:b:f': 0}
keys ['a:b:c', 'a:b:f']
key a:b:c
found : a:b:c
keys ['a', 'b', 'c']
first_key a , dict_ {'a:b:c': 'd', 'a:b:f': 0}
keys ['b:c']
key b:c
found : b:c
keys ['b', 'c']
first_key b , dict_ {'b:c': 'd'}
keys ['c']
key c
keys ['c']
key c
dict_[first_key] {'c': 'd'}
keys ['b:c']
key b:c
found : b:c
keys ['b', 'c']
first_key b , dict_ {'b:c': 'd'}
keys ['c']
key c
keys ['c']
key c
dict_[first_key] {'c': 'd'}
dict_[first_key] {'b': {'c': 'd'}}
key a:b:f
found : a:b:f
keys ['a', 'b', 'f']
first_key a , dict_ {'a:b:f': 0, 'a': {'b': {'c': 'd'}}}
keys ['b:f']
key b:f
found : b:f
keys ['b', 'f']
first_key b , dict_ {'b:f': 0}
keys ['f']
key f
keys ['f']
key f
dict_[first_key] {'f': 0}
dict_[first_key] {'b': {'c': 'd', 'f': 0}}
{'a': {'b': {'c': 'd', 'f': 0}}}


In [101]:
expand_keys(sweep_config)
pprint.pprint(sweep_config)

keys ['rewards', 'parameter_sets']
key rewards
keys ['G', 'D', 'D-Indirect-Step', 'D-Indirect-Traj', 'Fitness Critic', 'Alignment']
key G
keys ['rewards:asv_reward', 'rewards:which_critic']
key rewards:asv_reward
found : rewards:asv_reward
keys ['rewards', 'asv_reward']
first_key rewards , dict_ {'rewards:asv_reward': 'global', 'rewards:which_critic': 'none'}
keys ['asv_reward']
key asv_reward
keys ['asv_reward']
key asv_reward
dict_[first_key] {'asv_reward': 'global'}
key rewards:which_critic
found : rewards:which_critic
keys ['rewards', 'which_critic']
first_key rewards , dict_ {'rewards:which_critic': 'none', 'rewards': {'asv_reward': 'global'}}
keys ['which_critic']
key which_critic
dict_[first_key] {'asv_reward': 'global', 'which_critic': 'none'}
key D
keys ['rewards:asv_reward', 'rewards:which_critic']
key rewards:asv_reward
found : rewards:asv_reward
keys ['rewards', 'asv_reward']
first_key rewards , dict_ {'rewards:asv_reward': 'difference', 'rewards:which_critic': 'none'}
keys

In [83]:
d1 = {'a': {'b': {'c': 0}}}
d2 = {'a': {'b': {'d': 1}}}
d1.update(d2)
d1

{'a': {'b': {'d': 1}}}

In [25]:
# Build a dictionary of configs
# c['simple_a']['G'] = config

def merge_configs(base_config, rewards_dict):
    for r in rewards_dict:
        reward_dict = rewards_dict[r]
        for combined_keys in reward_dict:
            split_keys = combined_keys.split(':')
            print(split_keys)
            # Turn this into a proper dictionary
        # kis = k.split(':')
        # print(kis)
        # nd = {}
        # print(kis)
        # for ki, kin in zip(kis,kis[1:]):
        #     nd[ki] = kin
        #     nd = nd[ki]
        # print(nd)


c = {}
# for p in sweep_config['parameter_sets']:
p = 'simple_a'

c[p] = {}
merge_configs(base_config, sweep_config['rewards'])

['rewards', 'asv_reward']
['rewards', 'which_critic']
['rewards', 'asv_reward']
['rewards', 'which_critic']
['rewards', 'asv_reward']
['rewards', 'influence_type']
['influence_heuristic']
['rewards', 'which_critic']
['rewards', 'asv_reward']
['rewards', 'influence_type']
['influence_heuristic']
['rewards', 'which_critic']
['rewards', 'which_critic']
['rewards', 'which_critic']
